In [3]:
import cv2
import mediapipe as mp
import numpy as np
import os
from tensorflow.keras.models import load_model

# --- 1. Load Emotion Recognition Resources ---
# Ensure you have the necessary files: 'emotion_model_fer2013.h5', 'emotion_labels.npy', and 'haarcascade_frontalface_default.xml'
MODEL_FILE = 'emotion_model_fer2013.h5'
CASCADE_FILE = 'haarcascade_frontalface_default.xml'
IMAGE_SIZE = (48, 48)

try:
    emotion_model = load_model(MODEL_FILE, compile=False)
    face_classifier = cv2.CascadeClassifier(CASCADE_FILE)
    emotion_labels = np.load('emotion_labels.npy', allow_pickle=True)
    print("Emotion recognition model loaded successfully.")
except Exception as e:
    print(f"Error loading emotion recognition resources: {e}")
    emotion_model = None
    face_classifier = None
    emotion_labels = None

# --- 2. Initialize MediaPipe for Hand Recognition ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Finger tip landmarks indices for counting
FINGER_TIPS = [8, 12, 16, 20]
THUMB_TIP = 4

# --- 3. Finger Counting Function (Improved Logic) ---
def count_fingers(landmarks, handedness):
    finger_count = 0
    
    # The handedness is a ClassificationList object
    # We access the label directly from the first element of the classification list
    hand_label = handedness.classification[0].label

    # Check for the thumb based on which hand is detected
    if hand_label == 'Right':
        if landmarks[THUMB_TIP].x < landmarks[THUMB_TIP - 1].x:
            finger_count += 1
    elif hand_label == 'Left':
        if landmarks[THUMB_TIP].x > landmarks[THUMB_TIP - 1].x:
            finger_count += 1

    # Check for the other four fingers (Index, Middle, Ring, Pinky)
    for tip_id in FINGER_TIPS:
        tip = landmarks[tip_id]
        knuckle = landmarks[tip_id - 2]
        if tip.y < knuckle.y:
            finger_count += 1
            
    return finger_count

# --- 4. Main Live Stream Loop ---
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Webcam access failed. Exiting.")
    exit()

while cap.isOpened():
    success, image = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip the image for a mirror-like view
    image = cv2.flip(image, 1)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # --- Process for Emotion Recognition ---
    if emotion_model and face_classifier and emotion_labels is not None:
        gray_frame = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)
            roi_gray = gray_frame[y:y + h, x:x + w]
            cropped_img = cv2.resize(roi_gray, IMAGE_SIZE)
            model_input = np.expand_dims(np.expand_dims(cropped_img.astype('float32') / 255.0, axis=0), axis=-1)
            predictions = emotion_model.predict(model_input, verbose=0)
            predicted_emotion = emotion_labels[np.argmax(predictions[0])]
            cv2.putText(image, str(predicted_emotion), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # --- Process for Finger Counting ---
    results_hands = hands.process(image_rgb)
    total_finger_count = 0

    if results_hands.multi_hand_landmarks:
        # Use zip to iterate over both landmarks and handedness in parallel
        for hand_landmarks, handedness in zip(results_hands.multi_hand_landmarks, results_hands.multi_handedness):
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            landmarks = hand_landmarks.landmark
            finger_count_per_hand = count_fingers(landmarks, handedness)
            total_finger_count += finger_count_per_hand
            
    cv2.putText(image, f'Fingers: {total_finger_count}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow('Live Gesture and Emotion Recognition', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Emotion recognition model loaded successfully.
